In [ ]:
import pandas as pd
import numpy as np
import torch
from pytorch_pretrained_bert import BertTokenizer,BertModel
import logging
import difflib

`data_withGoldenLabel`为人工标注标签

`data_withSilverLabel`为算法标注标签

`data`去重:重复数据直接删除

将`author`中`nan`替换为`'Not Available'`

In [3]:
data = pd.read_table("./DataSet/book/book.txt" , sep='\t' , header=None , names=['source','isbn','name','author'])
data = data.drop_duplicates().reset_index(drop=True)
data.fillna('Not Available', inplace=True)
data['encode']=None
GoldenLabel = pd.read_table("./DataSet/book/book_golden.txt" , sep='\t' , header=None , names=['isbn','author'])
SilverLabel = pd.read_table("./DataSet/book/book_silver.txt" , sep='\t' , header=None , names=['isbn','author'])
SilverLabel = SilverLabel[SilverLabel['isbn']!='F']

加载`bert`模型和分词器

In [ ]:
logging.basicConfig(level=logging.INFO)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

此处逐条处理数据，后期考虑封装成`Processor`

将每条数据的三部分拼接向量化还是分别向量化？

选用`[CLS]`的最后一个隐层编码

GPU内存不足，暂定在CPU上运行

In [ ]:
for i in range(0,len(data)):
    item = data.loc[i]
    text = '[CLS] ' + (item['source']) + ' [SEP] ' + (item['name']) + ' [SEP] ' + (item['author']) + ' [SEP]'
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    #torch.cuda.empty_cache()
    #model.to('cuda')
    #tokens_tensor = tokens_tensor.to('cuda')
    with torch.no_grad():
        encoded_layers,_ = model(tokens_tensor)
    data.loc[i]['encode'] = encoded_layers[11][0][0]

In [86]:
data_golden

,source,isbn,name,author,encode,label
0,ProService,9780073516677,Computing Essentials,"O'Leary, Timothy J.; O'Leary, Linda I.","[tensor(-1.1033), tensor(-0.4017), tensor(-1.0...",True
1,Collegebooksdirect.com,9780073516677,Computing Essentials,"O'Leary, Timothy J.; O'Leary, Linda I.","[tensor(-0.9661), tensor(-0.4374), tensor(-1.0...",True
2,textbooksNow,9780073516677,Computing Essentials: Complete (2007),Oleary,"[tensor(-0.8278), tensor(-0.3013), tensor(-0.3...",False
3,www.textbooksrus.com,9780073516677,COMPUTING ESSENTIALS 2007:COMP,OLEARY,"[tensor(-0.8151), tensor(-0.1431), tensor(-0.2...",False
4,LGTextbooks.com,9780073516677,COMPUTING ESSENTIALS 2007 COMPLETE EDITION,O'LEARY,"[tensor(-0.8697), tensor(-0.1123), tensor(-0.2...",False
...,...,...,...,...,...,...
2176,Gunter Koppon,1555582745,Tru64 UNIX Troubleshooting - Diagnosing and Co...,Moore,"[tensor(-0.7027), tensor(-0.0370), tensor(-0.2...",False
2177,WorldOfBooks,1555582745,Tru64 UNIX Troubleshooting - Diagnosing and Co...,Moore,"[tensor(-0.5669), tensor(-0.1366), tensor(-0.2...",False
2178,Stratford Books,1555582745,Tru64 Unix Troubleshooting: Diagnosing and Cor...,"Moore, Martin (Author) Hancock, Steven (Author...","[tensor(-0.7188), tensor(-0.1007), tensor(-0.4...",False
2179,Gunars Store,1555582745,Tru64 UNIX Troubleshooting - Diagnosing and Co...,Moore,"[tensor(-0.6384), tensor(-0.2438), tensor(-0.2...",False


存储中间结果

In [ ]:
torch.save( data['encode'] , './DataSet/book/claims_encode.pt' )
data['encode'] = None
data.to_csv( './DataSet/book/claims.txt' , sep='\t' , index=False )

读取中间结果

In [ ]:
data = None
data = pd.read_csv( "./DataSet/book/claims.txt" , sep='\t' )
#n_data['source'] = n_data['source'].astype(str)
#n_data['isbn'] = n_data['isbn'].astype(str)
#n_data['name'] = n_data['name'].astype(str)
#n_data['author'] = n_data['author'].astype(str)
data['encode'] = torch.load("./DataSet/book/claims_encode.pt")
GoldenLabel = pd.read_table("./DataSet/book/book_golden.txt" , sep='\t' , header=None , names=['isbn','author'])
SilverLabel = pd.read_table("./DataSet/book/book_silver.txt" , sep='\t' , header=None , names=['isbn','author'])

这里讨巧使用`difflib`来度量字符串相似度，相似度的阈值后期可调，发现字符串肉眼相似但相似度量极低的情况！考虑第三方库的字符串相似度量？

后续可能需要改进，考虑再次引入`bert`

In [4]:
data['label'] = None
data['label'] = data['label'].astype(bool)
data_golden = pd.DataFrame(data.drop(data.index,inplace=False))

for index1,row1 in GoldenLabel.iterrows():
    str1 = row1['author'].lower()
    data_goldenSlice = pd.DataFrame(data[data['isbn']==row1['isbn']])
    for index2,row2 in data_goldenSlice.iterrows():
        str2 = row2['author'].lower()
        data_goldenSlice.loc[index2,'label'] = (difflib.SequenceMatcher(None,str1,str2).quick_ratio()>0.8)
    data_golden = data_golden.append(data_goldenSlice)

In [83]:
#使用sklearn的TfidfVectorizer生成特征向量来比较bert的效果，发现比bert好很多？
from sklearn.feature_extraction.text import TfidfVectorizer
data_golden.reset_index(drop=True,inplace=True)
vectorizer = TfidfVectorizer(min_df=0.005,max_df=0.8)
vectorizer.fit(data_golden['source'])
V1 = vectorizer.transform(data_golden['source']).toarray()
vectorizer.fit(data_golden['name'])
V2 = vectorizer.transform(data_golden['name']).toarray()
vectorizer.fit(data_golden['author'])
V3 = vectorizer.transform(data_golden['author']).toarray()
feature = np.concatenate((V1,V2,V3),axis=1)
feature = torch.Tensor(feature)
torch.save( feature , './DataSet/book/golden/claims_golden_encode_tfidf.pt' )

存储中间结果

In [ ]:
data_golden.reset_index(drop=True,inplace=True)
torch.save( data_golden['encode'] , './DataSet/book/golden/claims_golden_encode.pt' )
data_golden['encode'] = None
data_golden.to_csv( './DataSet/book/golden/claims_golden.txt' , sep='\t' , index=False )

读取中间结果

In [85]:
data_golden = pd.read_csv( './DataSet/book/golden/claims_golden.txt' , sep='\t' )
data_golden['encode'] = torch.load('./DataSet/book/golden/claims_golden_encode.pt')

book_silver.txt数据集有点问题，下一块代码暂且搁置

这个`book`数据集问题很大，值得吐槽；`dataset`中存在大量冗余数据；golden中存在两条`isbn`为`F`的奇怪数据；`silver`中许多数据`isbn`和`dataset`对应不上

In [ ]:
'''
data['label'] = None
data['label'] = data['label'].astype(bool)
data_silver = pd.DataFrame(data.drop(data.index,inplace=False))

for index1,row1 in SilverLabel.iterrows():
    str1 = row1['author'].lower()
    data_silverSlice = pd.DataFrame(data[data['isbn']==row1['isbn']])
    for index2,row2 in data_silverSlice.iterrows():
        str2 = row2['author'].lower()
        data_silverSlice.loc[index2,'label'] = (difflib.SequenceMatcher(None,str1,str2).quick_ratio()>0.8)
    data_silver = data_silver.append(data_silverSlice)
'''